* Global params:
    * n: No. of global samples
    * v_L: Global variance of logical failure rate estimator
    * p_L: Global estimator of logical failure rate
    * xi_min: Global path cutoff parameter, defines implicitely global w_max
    * eta: Global target variance (set either this or n?)
    * Protocol: Protocol object, containing:
        * C: Set of circuits defined in protocol
        * R(m,i): Rule function to calculate next circuit index i+1 from measurement result m and prev. circuit index i
    * A: Growing list of relevant weight-subset selection probabilities (i.e. binomial coefficients) per circuit in P.C
    * N: Matrix of no. of samples for each (circuit, weight) combination from C.
    * P: Matrix of transition probabilities between (circuit,weight) combinations.
    * V: Matrix of variances corresponding to each element in P.

In [20]:
import pecos as pc

class Protocol:
    
    def __init__(self, circuits):
        self.circuits = circuits
        self.i = 0 # circuit list index
    
    def next(self):
        """ Define rules for selecting next circuit. """
        NotImplemented
    
    def end(self):
        """ Define conditions for ending the protocol. """
        NotImplemented
        
class RUS(Protocol):
    """ Repeat-until-success protocol """
    
    def __init__(self, n, circuits):
        self.n = n
        super().__init__(circuits)
    
    def end(self, flag):
        if self.i < self.n or flag == 0:
            self.i += 1
            return False
        else:
            self.i = 0
            return True
        
    def next(self, flag):
        if not self.end(flag):
            return self.circuits[self.i]

class RepeatedGHZ(RUS):
    
    def __init__(self, n):
        ghz = pc.circuits.QuantumCircuit()
        ghz.append('H', {0})
        ghz.append('CNOT', {(0,1)})
        ghz.append('CNOT', {(1,2)})
        ghz.append('CNOT', {(2,3)})
        ghz.append('CNOT', {(3,4)})
        ghz.append('CNOT', {(0,4)})
        ghz.append('measure Z', {4})
        super().__init__(n, [ghz] * n)
        
p = RepeatedGHZ(3)
print(p.next(0))

QuantumCircuit([{'H': {0}}, {'CNOT': {(0, 1)}}, {'CNOT': {(1, 2)}}, {'CNOT': {(2, 3)}}, {'CNOT': {(3, 4)}}, {'CNOT': {(0, 4)}}, {'measure Z': {4}}])


In [ ]:
circ_runner = pc.circuit_runners.Standard(seed=42)


In [ ]:
import numpy as np

erv = lambda v0, v1: v1 - v0
var = lambda p, N, A: None

# Need method to define size of matrices: max_weight x max_len
max_weight = 4
max_len = 6
# A can be calculated in advance then.. but don't know possible circuit sequences in advance..